In [1]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

Using TensorFlow backend.


In [2]:
img_width, img_height = 16, 16

nb_epoch = 50
batch_size = 32

In [3]:
base_dir = '/home/niaki/Code/ImageNet/tiny-imagenet-200'

train_data_dir      = base_dir + '/tiny_train16'
validation_data_dir = base_dir + '/tiny_validation16'
test_data_dir       = base_dir + '/tiny_test16'

train_descrs_dir      = base_dir + '/tiny_sifts/tiny_train16'
validation_descrs_dir = base_dir + '/tiny_sifts/tiny_validation16'
test_descrs_dir       = base_dir + '/tiny_sifts/tiny_test16'

In [4]:
def loading_data(data_dir):
    files = listdir(data_dir + '/class0')
    files.sort()

    images = []

    for file in files:
        image = imageio.imread(data_dir + '/class0/' + file)
    #     image = np.expand_dims(image, axis=0)
        images.append(image)

    images = np.array(images)
    images = images.astype(np.float64) / 255
    print(images.shape)
    return images

In [5]:
x_train = loading_data(train_data_dir)
x_validation = loading_data(validation_data_dir)

(157086, 16, 16, 3)
(3932, 16, 16, 3)


In [6]:
def loading_descrs(descrs_dir):
    files = listdir(descrs_dir + '/class0')
    files.sort()

    descrs = []

    for file in files:
        descr = np.load(descrs_dir + '/class0/' + file)
        descrs.append(descr)

    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print(descrs.shape)
    return descrs

In [7]:
y_train = loading_descrs(train_descrs_dir)
y_validation = loading_descrs(validation_descrs_dir)

(157086, 128)
(3932, 128)


In [8]:
input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(128, activation="sigmoid")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', metrics=['accuracy'], loss='binary_crossentropy')

encoder.summary()

W0817 00:37:07.890718 140532395018048 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 00:37:07.911880 140532395018048 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 00:37:07.915203 140532395018048 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 00:37:07.926973 140532395018048 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0817 00:37:07.9620

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 16, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 32)          0         
__________

In [9]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [10]:
model_version = '0.1.5.3_sigmoid_adadelta_bce_generated'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, 32),
                steps_per_epoch=157086,
                epochs=50,
                validation_data=fixed_generator(x_validation, y_validation, 32),
                validation_steps=3932,
                callbacks=[checkpointer]
                )

encoder.save(base_dir + '/ae' + model_version + '.h5')

W0817 00:37:55.105670 140532395018048 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/50
157086/157086 [==============================] - 888s 6ms/step - loss: 0.1141 - acc: 0.7726 - val_loss: 0.1117 - val_acc: 0.7741
Epoch 2/50
157086/157086 [==============================] - 883s 6ms/step - loss: 0.1113 - acc: 0.7727 - val_loss: 0.1095 - val_acc: 0.7744
Epoch 3/50
157086/157086 [==============================] - 885s 6ms/step - loss: 0.1108 - acc: 0.7727 - val_loss: 0.1096 - val_acc: 0.7744
Epoch 4/50
157086/157086 [==============================] - 884s 6ms/step - loss: 0.1105 - acc: 0.7727 - val_loss: 0.1092 - val_acc: 0.7744
Epoch 5/50
157086/157086 [==============================] - 880s 6ms/step - loss: 0.1103 - acc: 0.7727 - val_loss: 0.1090 - val_acc: 0.7744
Epoch 6/50
157086/157086 [==============================] - 882s 6ms/step - loss: 0.1102 - acc: 0.7727 - val_loss: 0.1110 - val_acc: 0.7744
Epoch 7/50
157086/157086 [==============================] - 883s 6ms/step - loss: 0.1101 - acc: 0.7727 - val_loss: 0.1090 - val_acc: 0.7744
Epoch 8/50
157086/15

In [13]:
encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_validation16/class0/patch0000.png").reshape(1,16,16,3))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)